In [ ]:
#@title Packages
Packages = "Packages" #@param {type:"string"}
!pip install metpy
!pip install xarray
!pip install arrow
!pip install metpy
!pip install torch
!pip install setuptools-scm
!pip install bresenham
import torch
def format_pytorch_version(version):
  return version.split('+')[0]
 
TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
 
def format_cuda_version(version):
  return 'cu' + version.replace('.', '')
 
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/req_command.py", line 241,

In [1]:
#@title Import Packages
import os
import sys
import torch
from collections import OrderedDict
from scipy.spatial import distance
from torch_geometric.utils import dense_to_sparse, to_dense_adj
from geopy.distance import geodesic
from metpy.units import units
import metpy.calc as mpcalc
from bresenham import bresenham
import yaml
# import util
# from util import config
import numpy as np
from datetime import datetime
import arrow
import metpy.calc as mpcalc
from metpy.units import units
from torch.utils import data
from torch.utils.data import Dataset
import os
import sys
proj_dir = os.path.dirname(os.path.abspath('/content/drive/MyDrive/Delhi Lockdown/GNNDelhi'))
sys.path.append(proj_dir)
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import arrow
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
import pickle
import glob
import shutil
from prettytable import PrettyTable
# from model.cells import GRUCell
from torch.nn import Sequential, Linear, Sigmoid
from torch_scatter import scatter_add#, scatter_sub  # no scatter sub in lastest PyG
from torch.nn import functional as F
from torch.nn import Parameter
from torch_geometric.nn import GATConv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#@title Dataset_load
Dataset = "" #@param {type:"string"}
#py_file_location = "F:\\BTP\\FINAL DATASETS\\3 HOURS Data"
#sys.path.append(os.path.abspath(py_file_location))
##DelhiData_3hr_sample_V8_smooth.npy contains samples> 450, pm2.5 replaced by for each place averaged previous 8 sample and future 8 sample
Hour_3_sample=np.load("/content/drive/MyDrive/Dataset.npy", allow_pickle = True)
# wswd=np.load("/content/drive/MyDrive/Delhi Lockdown/GNNDelhi/DelhiData_3hr_sample_wswd.npy")
Hour_3_sample=np.float32(Hour_3_sample)

In [3]:
Hour_3_sample.shape

(8767, 266, 9)

0.   blh
1.   aod
2. kx
3. pm2.5
4. sp
5. t2m
6. tp
7. u10
8. v10


In [9]:
#@title Dataset_object
class HazeData(data.Dataset):
    def __init__(self,
                       hist_len=16,
                       pred_len=8,
                       dataset_num=1,
                       flag='T'):
 
        if flag == 'Train':
            start_time_str = [[2022, 1, 1], 'GMT']
            end_time_str = [[2022, 10, 23], 'GMT']
        elif flag == 'Val':
            start_time_str = [[2020, 6, 2], 'GMT']
            end_time_str = [[2022, 8, 6], 'GMT']
        elif flag == 'Test':
            start_time_str = [[2022, 10,24], 'GMT']
            end_time_str = [[2022,12,31], 'GMT']
        else:
            raise Exception('Wrong Flag!')
        self.start_time = self._get_time(start_time_str)
        self.end_time = self._get_time(end_time_str)
        self.data_start = self._get_time([[2022, 1, 1, 0, 0], 'GMT'])
        #config['dataset']['data_start']
        #config['dataset']['data_end']
        self.data_end = self._get_time([[2022, 12, 31, 11, 0], 'GMT'])
        # file_dir = config['filepath']
        # self.knowair_fp = file_dir
        # print(self.knowair_fp)
        #self.graph = graph
        self._load_npy()
        self._gen_time_arr()
        self._process_time()
        #self._process_feature2()
        self.feature = np.float32(self.feature)
        self.pm25 = np.float32(self.pm25)
       
        seq_len = hist_len + pred_len
        # print(seq_len)
        self._add_time_dim(seq_len)
        self._calc_mean_std()
        # self._norm()
        # self._get_time()
 
    def _norm(self):
        # print(np.isnan(self.feature))
        self.feature = (self.feature - self.feature_mean) / self.feature_std
        # print(np.isnan(self.feature))
        self.pm25 = (self.pm25 - self.pm25_mean) / self.pm25_std
        # print(self.pm25.shape)
 
 
    def _add_time_dim(self, seq_len):
 
        def _add_t(arr, seq_len):
            t_len = arr.shape[0]
            #print(t_len)
            #print(seq_len)
            assert t_len > seq_len
            arr_ts = []
            for i in range(seq_len, t_len):
                arr_t = arr[i-seq_len:i]
                arr_ts.append(arr_t)
            arr_ts = np.stack(arr_ts, axis=0)
            return arr_ts
        
        self.pm25 = _add_t(self.pm25, seq_len)
        # print(self.pm25.shape)
        self.feature = _add_t(self.feature, seq_len)
        self.time_arr = _add_t(self.time_arr, seq_len)
        # print("time se panga",self.time_arr.shape)
 
    def _calc_mean_std(self):
        self.featmean = self.feature.mean(axis=(0,1,2))
        # print("featmean",self.featmean.shape)
        self.featstd = self.feature.std(axis=(0,1,2))
        # print("featstd",self.featstd)
        # self.wind_mean = self.featmean[-2:]
        # self.wind_std = self.featstd[-2:]
        self.pm25_mean = self.pm25.mean()
        self.pm25_std = self.pm25.std()
        self.feature_mean = self.featmean
        self.feature_std = self.featstd
        # self.wind_mean = self.featmean[-2:]
        # self.wind_std = self.featstd[-2:]
        # self.pm25_mean = PM_mean
        # self.pm25_std = PM_std
 
    def _process_feature1(self):
        # metero_var = config['data']['metero_var']
        # # print(metero_var)
        # metero_use = config['experiments']['metero_use']
        # metero_idx = [metero_var.index(var) for var in metero_use]
        # # print(metero_idx)
        # self.feature = self.feature[:,:,metero_idx]
        print(self.feature.shape)
        # u = self.feature[:, :, -2] * units.meter / units.second
        # v = self.feature[:, :, -1] * units.meter / units.second
        # speed = 3.6 * mpcalc.wind_speed(u, v)._magnitude
        # direc = mpcalc.wind_direction(u, v)._magnitude
        # print(speed.shape)
        # speed =self.feature[:,:,2]
        # direc=self.feature[:,:,1]
        h_arr = []
        # print("fygj",h_arr)
        w_arr = []
        count1=0
        for i in self.time_arrow:
            # print("fygj",i)
            # count1=count1+1
            # print(count1)
            h_arr.append(i.hour)
            w_arr.append(i.isoweekday())
        h_arr = np.stack(h_arr, axis=-1)
        w_arr = np.stack(w_arr, axis=-1)
        h_cos=np.cos(2 * np.pi * h_arr/23.0)
        h_sin=np.sin(2 * np.pi * h_arr/23.0)
        w_cos=np.cos(2 * np.pi * w_arr/6.0)
        w_sin=np.sin(2 * np.pi * w_arr/6.0)
        # print("Age h_arr",w_arr.shape)
        #h_cos = np.repeat(h_cos[:, None], self.graph.node_num, axis=1)
        #w_cos = np.repeat(w_cos[:, None], self.graph.node_num, axis=1)
        #h_sin = np.repeat(h_sin[:, None], self.graph.node_num, axis=1)
        #w_sin = np.repeat(w_sin[:, None], self.graph.node_num, axis=1)
        # print("Pore h_arr",h_arr.shape)
        # print("warr",w_arr.shape)
        # print("featdim",self.feature.shape)
        self.feature = np.concatenate([self.feature, h_cos[:, :, None], w_cos[:, :, None], h_sin[:, :, None], w_sin[:, :, None]], axis=-1)
        # print("feature_clubbed",self.feature.shape)
    def _process_feature2(self):   
        h_arr = []
        w_arr = []
        count1=0
        for i in self.time_arrow:
            h_arr.append(i.hour)
            w_arr.append(i.isoweekday())     
        h_arr = np.stack(h_arr, axis=-1)
        w_arr = np.stack(w_arr, axis=-1)
        #h_arr = np.repeat(h_arr[:, None], self.graph.node_num, axis=1)
        #w_arr = np.repeat(w_arr[:, None], self.graph.node_num, axis=1)
        self.feature = np.concatenate([self.feature,h_arr[:,:,None],w_arr[:,:,None]],axis=-1)
    def _process_time(self):
        start_idx = self._get_idx(self.start_time)
        # print( "start",start_idx)
        end_idx = self._get_idx(self.end_time)
        # print("end",end_idx)
        self.pm25 = self.pm25[start_idx: end_idx+1, :]
        self.feature = self.feature[start_idx: end_idx+1, :]
        # print("feat_today",self.feature.shape)
        # print(np.isnan(self.feature))
        self.time_arr = self.time_arr[start_idx: end_idx+1]
        self.time_arrow = self.time_arrow[start_idx: end_idx + 1]
        # print(time_arrow.shape)
 
    def _gen_time_arr(self):
        self.time_arrow = []
        self.time_arr = []
        count=0
        for time_arrow in arrow.Arrow.interval('hours', self.data_start, self.data_end.shift(hours=+3), 3):
            # print(time_arrow.int_timestamp)
            # print(1)
            # count +=1
            # print("time_arr", count)
            self.time_arrow.append(time_arrow[0])
            self.time_arr.append(time_arrow[0].int_timestamp)
        self.time_arr = np.stack(self.time_arr, axis=-1)
        # print("time_arrow",self.time_arrow)
        # print("load",self.time_arr.shape)
    def _load_npy(self):
        # print("load",self.knowair_fp)
        # self.knowair = Hour_3_sample
        self.feature = np.delete(Hour_3_sample,3,2)
        # print("feattter",self.feature.shape)
        self.pm25 = Hour_3_sample[:,:,3]
        # print("self", self.pm25.type)
    def _get_idx(self, t):
        t0 = self.data_start
        return int((t.timestamp() - t0.timestamp()) / (60*60*3))
 
    def _get_time(self, time_yaml):
        arrow_time = arrow.get(datetime(*time_yaml[0]), time_yaml[1])
        return arrow_time
    def __len__(self):
        return len(self.pm25)
 
    def __getitem__(self, index):
        return self.pm25[index], self.feature[index], self.time_arr[index]
        # torch.tensor(list(self.pm25[index]),dtype=torch.float32) , torch.tensor(list(self.feature[index]),dtype=torch.float32), torch.tensor(list(self.time_arr[index]),dtype=torch.float32)
    
 
if __name__ == '__main__':
    # from graph import Graph
    #graph = Graph()
    train_data = HazeData(flag='Train')
    #val_data = HazeData(flag='Val')
    test_data = HazeData(flag='Test')

In [10]:
train_data.pm25

array([[[ 47.726665,  38.443333, 341.5     , ..., 149.      ,
         242.66667 ,  46.666668],
        [ 51.483334,  31.316668, 323.33334 , ..., 154.33333 ,
         284.33334 ,  53.666668],
        [ 63.036667,  29.228333, 340.      , ..., 132.33333 ,
         260.33334 ,  54.333332],
        ...,
        [ 33.746666,  38.633335,  63.75    , ...,  34.666668,
          52.333332,  66.      ],
        [ 21.55    ,  38.18667 ,  87.916664, ...,  48.666668,
         116.166664,  61.833332],
        [ 27.39    ,  28.191668,  15.833333, ...,  12.666667,
          19.      ,  60.333332]],

       [[ 51.483334,  31.316668, 323.33334 , ..., 154.33333 ,
         284.33334 ,  53.666668],
        [ 63.036667,  29.228333, 340.      , ..., 132.33333 ,
         260.33334 ,  54.333332],
        [ 56.656666,  17.651667, 132.16667 , ...,  96.      ,
         121.833336,  54.5     ],
        ...,
        [ 21.55    ,  38.18667 ,  87.916664, ...,  48.666668,
         116.166664,  61.833332],
        [ 27

In [11]:
#@title Data_Normalization
train_data.feature=(train_data.feature-train_data.feature_mean)/train_data.feature_std
train_data.pm25=(train_data.pm25-train_data.pm25_mean)/train_data.pm25_std
test_data.feature=(test_data.feature-train_data.feature_mean)/train_data.feature_std
test_data.pm25=(test_data.pm25-train_data.pm25_mean)/train_data.pm25_std


In [12]:
#@title LSTMcell

class LSTMCell(nn.Module):

    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(-1))
        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)
        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)
        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy, cy)

In [13]:
class LSTM(nn.Module):
    def __init__(self, hist_len, pred_len, in_dim, city_num, batch_size, device):
        super(LSTM, self).__init__()
        self.device = device
        self.hist_len = hist_len
        self.pred_len = pred_len
        self.city_num = city_num
        self.batch_size = batch_size
        self.in_dim = in_dim
        self.hid_dim =64
        self.m = nn.Dropout(p=0.2)
        self.out_dim = 1
        self.fc_in = nn.Linear(self.in_dim, self.hid_dim)
        self.fc_out = nn.Linear(self.hid_dim, self.out_dim)
        self.lstm_cell = LSTMCell(self.hid_dim, self.hid_dim)
 
    def forward(self, pm25_hist, feature):
        pm25_pred = []
        h0 = torch.zeros(self.batch_size * self.city_num, self.hid_dim).to(self.device)
        hn = h0
        c0 = torch.zeros(self.batch_size * self.city_num, self.hid_dim).to(self.device)
        cn = c0
        # xn = torch.unsqueeze(pm25_hist[:,-1],-1)
        # print("xn",xn.shape)
        # print("feat",feature.shape)
        for i in range(self.hist_len):
            # print(torch.unsqueeze(xn,-1).shape)
            # print(feature[:, self.hist_len+i].shape)
            x = torch.cat((torch.unsqueeze(pm25_hist[:,i],-1), feature[:, i]), dim=-1)
            # print(x.shape)
            x = self.fc_in(x)
            hn, cn = self.lstm_cell(x, (hn, cn))
            xn = hn.view(self.batch_size, self.city_num, self.hid_dim)
            xn = self.fc_out(xn)
  
        # xn = torch.unsqueeze(pm25_hist[:,-1],-1)
        for i in range(self.pred_len):
            # print(torch.unsqueeze(xn,-1).shape)
            # print(feature[:, self.hist_len+i].shape)
            x = torch.cat((xn, feature[:, self.hist_len+i]), dim=-1)
            # print(x.shape)
            x = self.fc_in(x)
            hn, cn = self.lstm_cell(x, (hn, cn))
            xn = hn.view(self.batch_size, self.city_num, self.hid_dim)
            xn = self.fc_out(xn)
            pm25_pred.append(xn)
        pm25_pred = torch.stack(pm25_pred, dim=1)
        pm25_pred = torch.squeeze(pm25_pred)
        return pm25_pred

In [14]:
#@title Training_module
import logging
lr = 0.0015 #@param {type:"number"}
#in_dim_lstm = 9
#in_dim_temp_gru = 32 #@param {type:"number"}
#in_dim_gnn_gru = 25 #@param {type:"number"}
weight_decay = 0.0001 #@param {type:"number"}
exp_repeat = 1 #@param {type:"number"}
epochs = 50 #@param {type:"number"}
in_dim = 9 #@param {type:"number"}
#save_numpy = True #@param {type:"raw"}
def get_logger(file_path):

        logger = logging.getLogger('gal')
        log_format = '%(asctime)s | %(message)s'
        formatter = logging.Formatter(log_format, datefmt='%m/%d %I:%M:%S %p')
        file_handler = logging.FileHandler(file_path)
        file_handler.setFormatter(formatter)
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)

        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)
        logger.setLevel(logging.INFO)
        return logger
torch.set_num_threads(1)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
 
# graph = Graph()
#city_num = graph.node_num
# s=np.concatenate((Cluster0,Cluster1,Cluster2),axis=1)
# s=np.squeeze(s)
# ss=np.argsort(s)
batch_size = 32
#epochs = 50

gradient_accumulations = 4
scaler = GradScaler()
hist_len = 16
pred_len = 8
#weight_decay = 0.0001
early_stop = 5
#lr = 0.0015
#in_dim_temp_gru=32
#in_dim_gnn_gru=25
results_dir = "/content/drive/MyDrive/Results"
dataset_num = 1
# exp_model = 'PM25_GNN'
# model= PM25_GNN
#exp_repeat = 5
save_npy = True
criterion = nn.MSELoss()
 
# print(config['dataset'][3]['train_start'])
# print('**************************')
# train_data = HazeData(graph, hist_len, pred_len, dataset_num, flag='Train')
# # val_data = HazeData(graph, hist_len, pred_len, dataset_num, flag='Val')
# test_data = HazeData(graph, hist_len, pred_len, dataset_num, flag='Test')
#in_dim = 14
#wind_mean, wind_std = train_data.wind_mean, train_data.wind_std
pm25_mean, pm25_std = train_data.pm25_mean, train_data.pm25_std
# print("wndmn",wind_mean.shape)
 
def get_metric(predict_epoch, label_epoch):
    haze_threshold = 60
    predict_epoch=predict_epoch[:,hist_len:]
    label_epoch=label_epoch[:,hist_len:]
    # print("pred",predict_epoch.shape)
    # print("lab",label_epoch.shape)
    predict_haze = predict_epoch >= haze_threshold
    predict_clear = predict_epoch < haze_threshold
    label_haze = label_epoch >= haze_threshold
    label_clear = label_epoch < haze_threshold
    hit = np.sum(np.logical_and(predict_haze, label_haze))
    miss = np.sum(np.logical_and(label_haze, predict_clear))
    falsealarm = np.sum(np.logical_and(predict_haze, label_clear))
    csi = hit / (hit + falsealarm + miss)
    pod = hit / (hit + miss)
    far = falsealarm / (hit + falsealarm)
    # predict = predict_epoch[:,:,:,0].transpose((0,2,1))
    # label = label_epoch[:,:,:,0].transpose((0,2,1))
    # predict = predict.reshape((-1, predict.shape[-1]))
    # label = label.reshape((-1, label.shape[-1]))
    #print("pred",predict_epoch.shape)
    #print("lab",label_epoch.shape)
    #print("pred",predict_epoch)
    mae = np.mean(np.mean(np.abs(predict_epoch - label_epoch), axis=1))
    rmse = np.mean(np.sqrt(np.mean(np.square(predict_epoch - label_epoch), axis=1)))
    predict_epoch=np.ndarray.flatten(predict_epoch)
    label_epoch=np.ndarray.flatten(label_epoch)
    max=np.ndarray.max(label_epoch)
    min=np.ndarray.min(label_epoch)
    nrmse=rmse/(max-min)
    # print("dsff",predict_epoch.shape)
    # mape = np.mean(np.abs((predict_epoch - label_epoch)/label_epoch), axis=1)
    r2=r2_score(label_epoch,predict_epoch)
    return rmse, nrmse, mae,r2, csi, pod, far

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
# def get_exp_info():
#     exp_info =  '============== Train Info ==============\n' + \
#                 'Dataset number: %s\n' % dataset_num + \
#                 'Train: %s --> %s\n' % (train_data.start_time, train_data.end_time) + \
#                 'Val: %s --> %s\n' % (val_data.start_time, val_data.end_time) + \
#                 'Test: %s --> %s\n' % (test_data.start_time, test_data.end_time) + \
#                 'City number: %s\n' % city_num + \
#                 'Use metero: %s\n' % config['experiments']['metero_use'] + \
#                 'batch_size: %s\n' % batch_size + \
#                 'epochs: %s\n' % epochs + \
#                 'hist_len: %s\n' % hist_len + \
#                 'pred_len: %s\n' % pred_len + \
#                 'weight_decay: %s\n' % weight_decay + \
#                 'early_stop: %s\n' % early_stop + \
#                 'lr: %s\n' % lr + \
#                 '========================================\n'
#     return exp_info
 
 
# def get_model():
#     if exp_model == 'MLP':
#         return MLP(hist_len, pred_len, in_dim)
#     elif exp_model == 'LSTM':
#         return LSTM(hist_len, pred_len, in_dim, city_num, batch_size, device)
#     elif exp_model == 'GRU':
#         return GRU(hist_len, pred_len, in_dim, city_num, batch_size, device)
#     elif exp_model == 'nodesFC_GRU':
#         return nodesFC_GRU(hist_len, pred_len, in_dim, city_num, batch_size, device)
#     elif exp_model == 'GC_LSTM':
#         return GC_LSTM(hist_len, pred_len, in_dim, city_num, batch_size, device, graph.edge_index)
#     elif exp_model == 'PM25_GNN':
#         return PM25_GNN(hist_len, pred_len, in_dim, city_num, batch_size, device, graph.edge_index, graph.edge_attr, wind_mean, wind_std)
#     elif exp_model == 'PM25_GNN_nosub':
#         return PM25_GNN_nosub(hist_len, pred_len, in_dim, city_num, batch_size, device, graph.edge_index, graph.edge_attr, wind_mean, wind_std)
#     else:
#         raise Exception('Wrong model name!')
 
 
def train(train_loader, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, data in tqdm(enumerate(train_loader)):
        pm25, feature, time_arr = data
        # print("training",pm25.shape)
        #print("featingdtype",feature.shape)
        pm25 = pm25.to(device)
        feature = feature.to(device)
        pm25_label = pm25[:, hist_len:]
        pm25_hist = pm25[:, :hist_len]
        with autocast():
             pm25_pred = model(pm25_hist, feature)
             #print("pred",pm25_pred.shape)
             #print("label",pm25_label.shape)
             loss= criterion(pm25_pred, pm25_label)
            #  loss2 = criterion(pred_gru, pm25_label) 
            #  loss=loss1+loss2
        # l2_lambda = 0.01
        # l2_reg = torch.tensor(0.).to(device)
        # for param in model.parameters():
        #     l2_reg += torch.norm(param)
        # loss += l2_lambda * l2_reg
        L1_reg = torch.tensor(0., requires_grad=True)
        for name, param in model.named_parameters():
           if 'weight' in name:
              L1_reg = L1_reg + torch.norm(param, 1)

        loss = loss + 10e-4 * L1_reg     
        scaler.scale(loss / gradient_accumulations).backward()
        optimizer.step()
        if (batch_idx + 1) % gradient_accumulations == 0:
              scaler.step(optimizer)
              scaler.update()
              optimizer.zero_grad()
        train_loss += loss.item()
    train_loss /= batch_idx + 1
    return train_loss
 
 
 
 
def test(test_loader, model):
    model.eval()
    predict_list = []
    label_list = []
    time_list = []
    test_loss = 0
    for batch_idx, data in enumerate(test_loader):
        pm25, feature, time_arr = data
        pm25 = pm25.to(device)
        feature = feature.to(device)
        pm25_label = pm25[:, hist_len:]
        pm25_hist = pm25[:, :hist_len]
        pm25_pred = model(pm25_hist, feature)
        loss = criterion(pm25_pred, pm25_label)
        # loss2 = criterion(pred_gru, pm25_label) 
        # loss=loss1+loss2
        test_loss += loss.item()
 
        pm25_pred_val = np.concatenate([pm25_hist.cpu().detach().numpy(), pm25_pred.cpu().detach().numpy()], axis=1) * pm25_std + pm25_mean
        pm25_label_val = pm25.cpu().detach().numpy() * pm25_std + pm25_mean
        predict_list.append(pm25_pred_val)
        label_list.append(pm25_label_val)
        time_list.append(time_arr.cpu().detach().numpy())
 
    test_loss /= batch_idx + 1
 
    predict_epoch = np.concatenate(predict_list, axis=0)
    label_epoch = np.concatenate(label_list, axis=0)
    time_epoch = np.concatenate(time_list, axis=0)
    predict_epoch[predict_epoch < 0] = 0
 
    return test_loss, predict_epoch, label_epoch, time_epoch
 
 
def get_mean_std(data_list):
    data = np.asarray(data_list)
    return data.mean(), data.std()
 
 
def main():
    # exp_info = get_exp_info()
    # print(exp_info)
 
    exp_time = arrow.now().format('YYYYMMDDHHmmss')
 
    train_loss_list, val_loss_list, test_loss_list, rmse_list, mae_list, csi_list, pod_list, far_list = [], [], [], [], [], [], [], []
 
    for exp_idx in range(exp_repeat):
        print('\nNo.%2d experiment ~~~' % exp_idx)
 
        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
        #val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
        test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)
 
        model= LSTM(hist_len, pred_len, in_dim, 266, batch_size, device)
        # model.load_state_dict(torch.load('/content/drive/MyDrive/Delhi_Model_Results/16_8/1/PM25GNN/20220103040118/00/13/model.pth'))
        # model.eval()
        model=model.to(device)
        print(str(model))
        count_parameters(model)
        #optimizer = torch.optim.Adam(model.parameters(), lr=0.0015)
        optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
 
        #exp_model_dir = os.path.join(results_dir, '%s_%s' % (hist_len, pred_len), str(dataset_num), 'PM25GNN', str(exp_time), '%02d' % exp_idx)
        exp_model_dir = os.path.join(results_dir, '%s_%s' % (hist_len, pred_len), 'SAGNN', str(exp_time), '%02d' % exp_idx)

        # if not os.path.exists(exp_model_dir):
        #     os.makedirs(exp_model_dir)
        # model_fp = os.path.join(exp_model_dir, 'model.pth')
        logger=get_logger(results_dir+'%02d'% exp_idx +'logger_SAGNN.log')
 
        val_loss_min = 100000
        best_epoch = 0
 
        # train_loss_, val_loss_ = 0, 0
        train_loss_=0
        epoch_rmse=[]
        epoch_mae=[]
        epoch_nrmse=[]
        epoch_r2=[]
        epoch_csi=[]
        epoch_pod=[]
        epoch_far=[]
        TrainLoss=[]
        TestLoss=[]
 
        for epoch in range(epochs):
            print('\nTrain epoch %s:' % (epoch))
 
            train_loss = train(train_loader, model, optimizer)
            # val_loss = val(val_loader, model)
 
            print('train_loss: %.4f' % train_loss)
            # print('val_loss: %.4f' % val_loss)
             
            test_loss, predict_epoch, label_epoch, time_epoch = test(test_loader, model)
                # print(test_loss,'#######################################')
            # train_loss_, val_loss_ = train_loss, val_loss
            train_loss_=  train_loss
            rmse, nrmse,mae,r2,csi, pod, far = get_metric(predict_epoch, label_epoch)
            
            print("rmse",rmse)
            epoch_rmse.append(rmse)
            epoch_mae.append(mae)
            epoch_nrmse.append(nrmse)
            epoch_r2.append(r2)
            epoch_csi.append(csi)
            epoch_pod.append(pod)
            epoch_far.append(far)
            TrainLoss.append(train_loss)
            TestLoss.append(test_loss)
            # if epoch - best_epoch > early_stop:
            #     break
 
            # if rmse < val_loss_min:
            # val_loss_min = rmse
            best_epoch = epoch
            exp_model_dir1= os.path.join(exp_model_dir,str(epoch))
            if not os.path.exists(exp_model_dir1):
                os.makedirs(exp_model_dir1)
            model_fp = os.path.join(exp_model_dir1, 'model.pth')
            # print('Minimum val loss!!!')
            torch.save(model.state_dict(), model_fp)
                # print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print('Save model: %s' % model_fp)

            # print('Train loss: %0.4f, Val loss: %0.4f, Test loss: %0.4f, RMSE: %0.2f, MAE: %0.2f,R2: %0.4f, CSI: %0.4f, POD: %0.4f, FAR: %0.4f' % (train_loss_, val_loss_, test_loss, rmse, mae,r2, csi, pod, far))
            print('Train loss: %0.4f, Test loss: %0.4f, RMSE: %0.2f,NRMSE: %0.4f, MAE: %0.2f,R2: %0.4f, CSI: %0.4f, POD: %0.4f, FAR: %0.4f' % (train_loss_, test_loss, rmse,nrmse, mae,r2, csi, pod, far))
            logger.info('Epoch: {}/{:.3f}---Train,{:.3f}----Test,RMSE{:.3f},NRMSE{:.4f},MAE{:.3f},R2{:.3f},CSI{:.3f},POD{:.3f},FAR{:.3f}\n'.format(epoch,train_loss_,test_loss,rmse,nrmse,mae,r2,csi,pod,far))

            if save_npy:
                np.save(os.path.join(exp_model_dir1, 'predict.npy'), predict_epoch)
                np.save(os.path.join(exp_model_dir1, 'label.npy'), label_epoch)
                np.save(os.path.join(exp_model_dir1, 'time.npy'), time_epoch)
        #if save_npy:
        np.save(os.path.join(exp_model_dir, 'epoch_rmse.npy'), np.array(epoch_rmse))
        np.save(os.path.join(exp_model_dir, 'epoch_mae.npy'), np.array(epoch_mae))
        np.save(os.path.join(exp_model_dir, 'epoch_nrmse.npy'), np.array(epoch_nrmse))
        np.save(os.path.join(exp_model_dir, 'epoch_r2.npy'), np.array(epoch_r2))
        np.save(os.path.join(exp_model_dir, 'epoch_csi.npy'), np.array(epoch_csi))
        np.save(os.path.join(exp_model_dir, 'epoch_pod.npy'), np.array(epoch_pod))
        np.save(os.path.join(exp_model_dir, 'epoch_far.npy'), np.array(epoch_far))
        np.save(os.path.join(exp_model_dir, 'epoch_trainloss.npy'), np.array(TrainLoss))
        np.save(os.path.join(exp_model_dir, 'epoch_testloss.npy'), np.array(TestLoss))
      #test_loss=val_loss_
        train_loss_list.append(train_loss_)
        # val_loss_list.append(val_loss_)
        test_loss_list.append(test_loss)
        rmse_list.append(rmse)
        mae_list.append(mae)
        csi_list.append(csi)
        pod_list.append(pod)
        far_list.append(far)
        #if save_npy: 
        np.save(os.path.join(exp_model_dir, 'trainlosslist.npy'), np.array(train_loss_list))
        np.save(os.path.join(exp_model_dir, 'test_loss_list.npy'), np.array(test_loss_list))
        np.save(os.path.join(exp_model_dir, 'rmse_list.npy'), np.array(rmse_list))
        np.save(os.path.join(exp_model_dir, 'mae_list.npy'), np.array(mae_list))
        np.save(os.path.join(exp_model_dir, 'csi_list.npy'), np.array(csi_list))
        np.save(os.path.join(exp_model_dir, 'pod_list.npy'), np.array(pod_list))
        np.save(os.path.join(exp_model_dir, 'far_list.npy'), np.array(far_list))
        # plot1=plt.figure(1)
        # plt.plot(epoch_rmse, label='RMSE')
        # plt.xlabel('Epochs')
        # plt.ylabel('RMSE(in μg/m3)')
        # file='RMSE.pdf'
        # plt.savefig(os.path.join(exp_model_dir1,file))
        # plt.clf()
        # plot2=plt.figure(2)
        # plt.plot(epoch_mae, label='MAE')
        # plt.xlabel('Epochs')
        # plt.ylabel('MAE(in μg/m3)')
        # file='MAE.pdf'
        # plt.savefig(os.path.join(exp_model_dir1,file))
        # plt.clf() 
        # plot3=plt.figure(3)
        # plt.plot(epoch_rmse, label='RMSE')
        # plt.plot(epoch_mae, label='MAE')
        # plt.xlabel('Epochs')
        # plt.ylabel('(in μg/m3)')
        # file='RMSE-MAE.pdf'
        # plt.savefig(os.path.join(exp_model_dir1,file))
        # plt.clf()
        # plot4=plt.figure(4)
        # plt.plot(TrainLoss, label='Train_loss')
        # plt.plot(TestLoss, label='Test_loss')
        # plt.xlabel('Epochs')
        # plt.ylabel('Loss')
        # file='Train_TestLoss.pdf'
        # plt.savefig(os.path.join(exp_model_dir1,file))
        # plt.clf() 
        print('\nNo.%2d experiment results:' % exp_idx)
        print(
            'Train loss: %0.4f, Test loss: %0.4f, RMSE: %0.2f, NRMSE:%0.2f ,MAE: %0.2f, R2: %0.2f, CSI: %0.4f, POD: %0.4f, FAR: %0.4f' % (
            train_loss_, test_loss, rmse,nrmse, mae,r2, csi, pod, far))
 
    exp_metric_str = '---------------------------------------\n' + \
                     'train_loss | mean: %0.4f std: %0.4f\n' % (get_mean_std(train_loss_list)) + \
                     'test_loss  | mean: %0.4f std: %0.4f\n' % (get_mean_std(test_loss_list)) + \
                     'RMSE       | mean: %0.4f std: %0.4f\n' % (get_mean_std(rmse_list)) + \
                     'MAE        | mean: %0.4f std: %0.4f\n' % (get_mean_std(mae_list)) + \
                     'CSI        | mean: %0.4f std: %0.4f\n' % (get_mean_std(csi_list)) + \
                     'POD        | mean: %0.4f std: %0.4f\n' % (get_mean_std(pod_list)) + \
                     'FAR        | mean: %0.4f std: %0.4f\n' % (get_mean_std(far_list))+\
                     'learningRate... %0.4f' % (lr) +\
                     'weight_decay %0.4f' % (weight_decay)
 
    metric_fp = os.path.join(os.path.dirname(exp_model_dir), 'metric.txt')
    with open(metric_fp, 'w') as f:
        f.write(str(model))
        f.write(exp_metric_str)
 
    print('=========================\n')
    print(exp_metric_str)
    print(str(model))
    print(metric_fp)
 
 
if __name__ == '__main__':
    main()

/usr/local/lib/python3.9/dist-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")



No. 0 experiment ~~~
LSTM(
  (m): Dropout(p=0.2, inplace=False)
  (fc_in): Linear(in_features=9, out_features=64, bias=True)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (lstm_cell): LSTMCell(
    (x2h): Linear(in_features=64, out_features=256, bias=True)
    (h2h): Linear(in_features=64, out_features=256, bias=True)
  )
)
+----------------------+------------+
|       Modules        | Parameters |
+----------------------+------------+
|     fc_in.weight     |    576     |
|      fc_in.bias      |     64     |
|    fc_out.weight     |     64     |
|     fc_out.bias      |     1      |
| lstm_cell.x2h.weight |   16384    |
|  lstm_cell.x2h.bias  |    256     |
| lstm_cell.h2h.weight |   16384    |
|  lstm_cell.h2h.bias  |    256     |
+----------------------+------------+
Total Trainable Params: 33985

Train epoch 0:


73it [01:59,  1.63s/it]


train_loss: 1.0681


04/26 08:36:23 PM | Epoch: 0/1.068---Train,1.091----Test,RMSE39.071,NRMSE0.0426,MAE33.982,R20.467,CSI0.732,POD0.864,FAR0.173

04/26 08:36:23 PM | Epoch: 0/1.068---Train,1.091----Test,RMSE39.071,NRMSE0.0426,MAE33.982,R20.467,CSI0.732,POD0.864,FAR0.173

INFO:gal:Epoch: 0/1.068---Train,1.091----Test,RMSE39.071,NRMSE0.0426,MAE33.982,R20.467,CSI0.732,POD0.864,FAR0.173



...

rmse 34.329617
Save model: /content/drive/MyDrive/Results/16_8/SAGNN/20230426203411/00/48/model.pth
Train loss: 0.5115, Test loss: 0.9303, RMSE: 34.33,NRMSE: 0.0375, MAE: 29.10,R2: 0.5453, CSI: 0.7399, POD: 0.8635, FAR: 0.1621

Train epoch 49:


0it [00:00, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
73it [01:55,  1.59s/it]


train_loss: 0.5138


04/26 10:21:23 PM | Epoch: 49/0.514---Train,0.881----Test,RMSE33.682,NRMSE0.0368,MAE28.400,R20.569,CSI0.714,POD0.801,FAR0.132

04/26 10:21:23 PM | Epoch: 49/0.514---Train,0.881----Test,RMSE33.682,NRMSE0.0368,MAE28.400,R20.569,CSI0.714,POD0.801,FAR0.132

INFO:gal:Epoch: 49/0.514---Train,0.881----Test,RMSE33.682,NRMSE0.0368,MAE28.400,R20.569,CSI0.714,POD0.801,FAR0.132



rmse 33.68239
Save model: /content/drive/MyDrive/Results/16_8/SAGNN/20230426203411/00/49/model.pth
Train loss: 0.5138, Test loss: 0.8807, RMSE: 33.68,NRMSE: 0.0368, MAE: 28.40,R2: 0.5695, CSI: 0.7137, POD: 0.8010, FAR: 0.1324

No. 0 experiment results:
Train loss: 0.5138, Test loss: 0.8807, RMSE: 33.68, NRMSE:0.04 ,MAE: 28.40, R2: 0.57, CSI: 0.7137, POD: 0.8010, FAR: 0.1324

---------------------------------------
train_loss | mean: 0.5138 std: 0.0000
test_loss  | mean: 0.8807 std: 0.0000
RMSE       | mean: 33.6824 std: 0.0000
MAE        | mean: 28.3995 std: 0.0000
CSI        | mean: 0.7137 std: 0.0000
POD        | mean: 0.8010 std: 0.0000
FAR        | mean: 0.1324 std: 0.0000
learningRate... 0.0015weight_decay 0.0001
LSTM(
  (m): Dropout(p=0.2, inplace=False)
  (fc_in): Linear(in_features=9, out_features=64, bias=True)
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (lstm_cell): LSTMCell(
    (x2h): Linear(in_features=64, out_features=256, bias=True)
    (h2h): Linear(